In [1]:
from pathlib import Path
from kedro.framework.session import KedroSession
from kedro.framework.session.session import _activate_session
from kedro.framework.startup import _add_src_to_path

current_dir = Path.cwd()  # this points to 'notebooks/' folder
project_path = current_dir.parent  # point back to the root of the project
_add_src_to_path(project_path/"src", project_path)

session = KedroSession.create("csmr_kedro", project_path)
_activate_session(session)
context = session.load_context()
catalog = context.catalog


/home/matheus/.local/lib/python3.8/site-packages/pkg_resources/__init__.py:1130: DeprecationWarning: Use of .. or absolute path in a resource path is not allowed and will raise exceptions in a future release.
  return get_provider(package_or_requirement).get_resource_filename(


2021-12-26 22:35:58,637 - root - INFO - Registered hooks from 2 installed plugin(s): kedro-mlflow-0.7.2
2021-12-26 22:35:58,639 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.


In [2]:
catalog.load('torch_model')

2021-12-26 22:35:59,392 - kedro.io.data_catalog - INFO - Loading data from `torch_model` (TorchDataSet)...
{'layer_norm': True, 'drop': 0.2, 'seq_length': 128, 'use_company_att': False, 'predict_company': True, 'company_alpha': 0.2}


Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


DataSetError: Failed while loading data from data set TorchDataSet(filepath=/home/matheus/kedro_projects/csmr-kedro/data/06_models/bert_sa.pt, load_args={'model_params': {'company_alpha': 0.2, 'drop': 0.2, 'layer_norm': True, 'predict_company': True, 'seq_length': 128, 'use_company_att': False}}, protocol=file, save_args={}).
Error(s) in loading state_dict for BertSA:
	Missing key(s) in state_dict: "bert.embeddings.position_ids", "bert.embeddings.word_embeddings.weight", "bert.embeddings.position_embeddings.weight", "bert.embeddings.token_type_embeddings.weight", "bert.embeddings.LayerNorm.weight", "bert.embeddings.LayerNorm.bias", "bert.encoder.layer.0.attention.self.query.weight", "bert.encoder.layer.0.attention.self.query.bias", "bert.encoder.layer.0.attention.self.key.weight", "bert.encoder.layer.0.attention.self.key.bias", "bert.encoder.layer.0.attention.self.value.weight", "bert.encoder.layer.0.attention.self.value.bias", "bert.encoder.layer.0.attention.output.dense.weight", "ber